---

# Notebook : Stacking avancé avec XGBoost, LightGBM, et optimisation des hyperparamètres

**Objectifs :**
1. Construire un modèle de stacking combinant **Random Forest**, **XGBoost**, et **LightGBM** comme modèles de base pour tirer parti des forces de chaque algorithme.
2. Utiliser un meta-modèle de **régression linéaire** pour la prédiction finale.
3. Optimiser les hyperparamètres de chaque modèle de base pour maximiser les performances.
4. Évaluer les performances globales du modèle de stacking sur le jeu de test.

---

## Étape 1 : Importer les bibliothèques nécessaires

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np
import joblib
import os

---

## Étape 2 : Charger les données et préparer les variables

In [2]:
# Charger les données
file_path = 'note-books/phase_2_training/04_correlation_analysis/data_final_features.csv'
df = pd.read_csv(file_path)

# Encoder `nutriscore_grade` en valeurs numériques
df['nutriscore_grade_encoded'] = df['nutriscore_grade'].map({'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4})

# Sélectionner les variables pertinentes
X = df[['fat_100g', 'saturated-fat_100g', 'energy-kcal_100g', 'sugars_100g', 'salt_100g', 'proteins_100g']]
y = df['nutriscore_grade_encoded']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

---

## Étape 3 : Configurer les modèles de base et le meta-modèle

In [3]:
# Modèles de base avec des hyperparamètres à optimiser
base_models = [
    ('random_forest', RandomForestRegressor(random_state=42)),
    ('xgboost', XGBRegressor(random_state=42)),
    ('lightgbm', LGBMRegressor(random_state=42))
]

# Meta-modèle : régression linéaire
meta_model = LinearRegression()

# Configurer le Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model, n_jobs=-1)

---

## Étape 4 : Optimiser les hyperparamètres des modèles de base

In [4]:
# Définir la grille des hyperparamètres pour chaque modèle de base
param_grid = {
    'random_forest__n_estimators': [100, 200, 300],
    'random_forest__max_depth': [None, 10, 20],
    'xgboost__n_estimators': [100, 200],
    'xgboost__max_depth': [3, 5],
    'xgboost__learning_rate': [0.01, 0.1],
    'lightgbm__n_estimators': [100, 200],
    'lightgbm__max_depth': [10, 20],
    'lightgbm__learning_rate': [0.01, 0.1]
}

# Configurer RandomizedSearchCV pour optimiser les hyperparamètres
random_search = RandomizedSearchCV(
    estimator=stacking_regressor,
    param_distributions=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    n_iter=10,  # Limite à 10 itérations pour réduire le temps de calcul
    random_state=42,
    n_jobs=-1,
    verbose=2
)

# Entraîner avec RandomizedSearchCV
random_search.fit(X_train, y_train)

# Meilleurs paramètres
best_params = random_search.best_params_
print("Meilleurs paramètres trouvés :", best_params)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] [LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009429 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1510
Total Bins 1515
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1513
[LightGBM] [Info] Number of data points in the train set: 89098, number of used features: 6
[LightGBM] [Info] Number of data points in the train set: 89098, number of used features: 6
[LightGBM] [Info] Number of data points

/Users/mwrevent/Dropbox/Mon Mac (Romualdraphique)/Desktop/code/Simplon/brief/Projet-OpenFood/documents_other/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END lightgbm__learning_rate=0.1, lightgbm__max_depth=20, lightgbm__n_estimators=100, random_forest__max_depth=20, random_forest__n_estimators=300, xgboost__learning_rate=0.01, xgboost__max_depth=5, xgboost__n_estimators=100; total time= 6.6min
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1510
[LightGBM] [Info] Number of data points in the train set: 89098, number of used features: 6
[LightGBM] [Info] Start training from score 2.205953
[CV] END lightgbm__learning_rate=0.1, lightgbm__max_depth=10, lightgbm__n_estimators=200, random_forest__max_depth=10, random_forest__n_estimators=300, xgboost__learning_rate=0.01, xgboost__max_depth=3, xgboost__n_estimators=100; total time= 4.1min
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001509 seconds.
You can set `force_row_wise=true` to remove the overhead.

---

## Étape 5 : Évaluer les performances du modèle optimisé

In [5]:
# Meilleur modèle de stacking optimisé
best_stacking_model = random_search.best_estimator_

# Évaluer sur le jeu de test
y_pred = best_stacking_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Évaluation des performances du modèle de stacking optimisé :")
print(f"Mean Squared Error (MSE) : {mse}")
print(f"Coefficient de détermination (R²) : {r2}")

Évaluation des performances du modèle de stacking optimisé :
Mean Squared Error (MSE) : 0.34801475436900686
Coefficient de détermination (R²) : 0.8074102113794162


---

## Étape 6 : Sauvegarder le modèle et les hyperparamètres

In [6]:
# Chemin de sauvegarde
output_folder = 'note-books/phase_2_training/09_stacking_advanced'
os.makedirs(output_folder, exist_ok=True)

# Sauvegarder le modèle de stacking optimisé
model_path = os.path.join(output_folder, 'stacking_advanced_optimized.joblib')
joblib.dump(best_stacking_model, model_path)

# Sauvegarder les meilleurs paramètres dans un fichier JSON
import json
hyperparams_path = os.path.join(output_folder, 'best_hyperparameters.json')
with open(hyperparams_path, 'w') as f:
    json.dump(best_params, f)

print(f"Modèle de stacking avancé optimisé sauvegardé sous : {model_path}")
print(f"Meilleurs hyperparamètres sauvegardés sous : {hyperparams_path}")

Modèle de stacking avancé optimisé sauvegardé sous : note-books/phase_2_training/09_stacking_advanced/stacking_advanced_optimized.joblib
Meilleurs hyperparamètres sauvegardés sous : note-books/phase_2_training/09_stacking_advanced/best_hyperparameters.json



---

### Explication des étapes

1. **Modèles de base et meta-modèle** : Utilisation de **Random Forest**, **XGBoost**, et **LightGBM** pour capturer divers aspects des données, avec un modèle de **régression linéaire** comme meta-modèle pour la prédiction finale.
2. **Optimisation des hyperparamètres** : Utilisation de **RandomizedSearchCV** pour explorer efficacement les hyperparamètres.
3. **Évaluation et sauvegarde** : Calcul des performances sur le jeu de test et sauvegarde du modèle optimisé et des hyperparamètres pour des analyses futures.

---

### Résumé

Dans ce notebook, nous avons :
1. Construit un modèle de **stacking avancé** avec des modèles de base diversifiés.
2. Optimisé les hyperparamètres pour maximiser les performances.
3. Évalué et sauvegardé le modèle optimisé et les hyperparamètres pour des analyses ultérieures ou un déploiement.